Test

In [23]:
!wget "https://raw.githubusercontent.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/master/src/datasets/exploration_datasets/gold/qa_dataset_col_mex_news_squad2.json" -O "qa_dataset_col_mex_news_squad2.json"

--2025-10-24 02:02:14--  https://raw.githubusercontent.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/master/src/datasets/exploration_datasets/gold/qa_dataset_col_mex_news_squad2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133 [text/plain]
Saving to: ‘qa_dataset_col_mex_news_squad2.json’

qa_dataset_col_mex_ 100%[===================>]     133  --.-KB/s    in 0s      

2025-10-24 02:02:14 (3.06 MB/s) - ‘qa_dataset_col_mex_news_squad2.json’ saved [133/133]



In [28]:
!ls -lh "/content/drive/MyDrive/qa_dataset_col_mex_news_squad2.json"
!head -n 10 "/content/drive/MyDrive/qa_dataset_col_mex_news_squad2.json"

-rw------- 1 root root 133 Oct 24 02:01 /content/drive/MyDrive/qa_dataset_col_mex_news_squad2.json
version https://git-lfs.github.com/spec/v1
oid sha256:7103b65b3b45ee4d63d57681fceae6df1123a839f483467f74a63583f3b6e14e
size 20171386


In [30]:
!pip install transformers evaluate torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00


In [31]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import evaluate
import json
from tqdm import tqdm

In [32]:
import torch
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

GPU available: False


In [27]:
import json

with open("/content/qa_dataset_col_mex_news_squad2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"✅ Loaded {len(data)} records successfully!")


✅ Loaded 1 records successfully!


In [ ]:
model_name = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"

qa_pipeline = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=model_name
)